# 1. Introduction

Baltimore, Maryland is infamous for its significantly high crime rate. Violent crime spiked in 2015 after the death of Freddie Gray, which touched off riots and has resulted in the huge increasing in murders. Baltimore recorded 344 homicides in 2015. To put that in perspective, New York City, a city with a 2015 population of 8,491,079, recorded a total of 339 homicides in 2015. Baltimore had a 2015 population of 621,849; which means that in 2015 Baltimore had a homicide rate 14 times higher than New York City's.

However, Baltimore is also a tourism city with beautiful harbor and delicious seafood. The city hosted about 25 million visitors a year, which is about 8% of the population in U.S. In order to give a better guidance to the visitors, this project will focus on the analysis crime in Baltimore and will find out the safest area in Baltimore with view suggestions according to the Foursquare location data.

# 2. Data Aquisition and Data Cleaning

## 2.1 Data Aquisition

The data for this project are two sorces. The first dataset is the [crime in Baltimore for 2012-2017](https://www.kaggle.com/sohier/crime-in-baltimore/kernels) from [Kaggle](https://www.kaggle.com/). This data presents all the crimes recorded from 2012 to 2017 in Baltimore. This data set contains the following __columns__:

* **CrimeDate**: Indicate the crime data in form of MM/DD/YYYY
* **CrimeTime**: Indicate the crime time in form of HH/MM/SS
* **CrimeCode**: Indicate the crime code of the crime event
* **Location**: Indicate the street location of the crime event
* **Description**: Indicate the type of crime, eg: Robbery/Shooting...
* **Inside/Outside**: Indicate whether the crime event happend inside or outside
* **Weapon**: Indicate the weapon used for the crime event
* **Post**: Not clear
* **District**: Indicate the district of the crime happened
* **Neighborhood**: Indicate the neighborhood of the crime happened
* **Longitude**: Coordinates Information
* **Latitude**: Coordinates Information
* **Location 1**: In form of (Latitude, Longitude)
* **Premise**: Indicate the type of location where the crime event happened.
* **Total incidents**: Indicate the number of incidents under this record (usually equals to 1)

The second dataset is the Foursquare location data which contains the venue information grouped by neighborhood. This data set contains the following **columns**:

* **Neighborhood**: The name of neighborhood
* **Neighborhood Latitude**: Coordinates information
* **Neighborhood Longitude**: Coordinated information
* **Venue**: The name of venue
* **Venue Latitude**: Venue coordinates information
* **Venue Longitude**: Venue coordinates information
* **Venue Category**: The type of venue